### <div align="center">***API REQUEST FOR DATABASE***</div>
***
Extracción de los registros relativos a los distritos 7,8,9: **Chamberí**, **Fuencarral-El Pardo**, **Moncloa-Aravaca**.

In [263]:
## CREDENCIALES PARA PODER USAR LA API

api_key= ""

### **IMPORTS**

In [264]:
from yelpapi import YelpAPI
import pandas as pd
import requests
import folium
import json

yelp_api = YelpAPI(api_key)

from utils import distritos

## CHAMBERI (6) : FUENCARRAL (7) : MONCLOA (8)
LATITUD = distritos[20]["latitude"]
LONGITUD = distritos[20]["longitude"]
RADIO = 1000

### **EXTRACCION CON LA API**

In [265]:
## CONFIGURACIÓN PARA QUE PANDAS MUESTRE LAS COLUMNAS DEL DF SIN TRUNCARLAS
pd.set_option('display.max_columns', None) 

In [266]:
response.status_code

200

In [267]:
url = 'https://api.yelp.com/v3/businesses/search'

headers = {
    'Authorization': 'Bearer ' + api_key
}

res = []
for dis in range(1):
    params = {    
    "latitude": LATITUD,  # Latitud del epicentro de búsqueda
    "longitude": LONGITUD,  # Longitud del epicentro de búsqueda
    "radius": LATITUD,  # Radio en metros para buscar restaurantes alrededor de la ubicación
    'categories': 'restaurants',  # Seleccionamos la categoría de comercio a extraer
    'limit': 50,  # Número de resultados que recupera (max 50
    }

    response = requests.get(url, headers=headers, params=params) # GET request
    if response.headers.get("Content-Type") == "application/json":
        try:
            # Parse the JSON response
            data = response.json()

            # Extract and process restaurants
            restaurants = data['businesses']        
                
            for restaurant in restaurants:
                id = restaurant['id']
                name = restaurant["name"]
                categories = restaurant['categories']
                rating = restaurant['rating']    
                address = ', '.join(restaurant['location']['display_address'])
                description = ""
                
                ## ALIAS
                if "alias" in restaurant:
                    alias = restaurant["alias"]
                else:
                    alias = None

                ## URL IMAGEN
                if "image_url" in restaurant:
                    image_url = restaurant["image_url"]
                else:
                    image_url = None

                ## URL NEGOCIO
                if "url" in restaurant:
                    url = restaurant["url"]
                else:
                    url = None

                ## CLOSED
                if "is_closed" in restaurant:
                    is_closed = restaurant["is_closed"]
                else:
                    is_closed = None

                ## REVIEW_COUNT
                if "review_count" in restaurant:
                    review_count = restaurant["review_count"]
                else:
                    review_count = None
                
                ## COORDINATES
                if "coordinates" in restaurant:
                    latitude = restaurant["coordinates"]["latitude"]
                    longitude = restaurant["coordinates"]["longitude"]
                else:
                    latitude = None
                    longitude = None
            
                ## PHONE
                if "phone" in restaurant:
                    phone = restaurant["phone"]
                else:
                    phone = None

                ## PRICE
                if "price" in restaurant:
                    price = restaurant["price"]
                else:
                    price = None 

                ## HOURS   
                if "hours" in restaurant:
                    hours = restaurant["hours"]
                else:
                    hours = None
                
                ## TRANSACTIONS
                if "transactions" in restaurant:
                    transactions = restaurant["transactions"]
                else:
                    transactions = None

                ## LANGUAGES
                if "languages" in restaurant:
                    languages = restaurant["languages"]
                else:
                    languages = None
                
                ## ATRIBUTES
                if "attributes" in restaurant:
                    attributes = restaurant["attributes"]
                else:
                    attributes = None

                ## TRANSIT
                if "transit" in restaurant:
                    transit = restaurant["transit"]
                else:
                    transit = None

                ## BEST TIME
                if "best_time" in restaurant:
                    best_time = restaurant["best_time"]
                else:
                    best_time = None

                ## DELIVERY
                if "delivery" in restaurant:
                    delivery = restaurant["delivery"]
                else:
                    delivery = None

                ## PICKUP
                if "pickup" in restaurant:
                    pickup = restaurant["pickup"]
                else:
                    pickup = None

                ## ONLINE ORDER
                if "online_ordering" in restaurant:
                    online_ordering = restaurant["online_ordering"]
                else:
                    online_ordering = None
                        
                ## PRIMEROS 5 REVIEWS 
                for c in categories:
                    description = description + c["title"] + " "
                description = description  

                # Make the API call to retrieve reviews for the current business
                reviews_response = yelp_api.reviews_query(id=restaurant["id"],location="Madrid", lang="es")
                        
                # Extract the reviews from the response
                reviews = reviews_response['reviews']
                        
                # Process and print the reviews
                        
                rev = ""
                for review in reviews[:5]:
                    review_text = review['text']
                    rating = review['rating']
                    reviewer_name = review['user']['name']

                    rev = rev + review_text + ". "   
                        
                
                res.append({
                        "id" : id,
                        "name" : name, 
                        "alias":alias, 
                        "categories": categories,
                        "description": description,
                        "latitude":latitude,
                        "longitude": longitude, 
                        "address": address,
                        "phone":phone,
                        "rating": rating,
                        "review_count":review_count,
                        "review": rev,
                        "url":url,
                        "image_url":image_url,
                        "is_closed":is_closed,
                        "price":price,
                        "hours":hours,
                        "transactions":transactions,
                        "languages":languages,
                        "attributes":attributes,
                        "transit":transit,
                        "best_time":best_time,
                        "delivery":delivery,
                        "pickup":pickup,
                        "online_ordering":online_ordering
                })
        except json.JSONDecodeError:
            print("La respuesta no es un JSON válido")

df_distrito = pd.DataFrame(res)

In [268]:
print(len(df_distrito))
df_distrito.head(3)

50


,id,name,alias,categories,description,latitude,longitude,address,phone,rating,review_count,review,url,image_url,is_closed,price,hours,transactions,languages,attributes,transit,best_time,delivery,pickup,online_ordering
0,GBDHMbAOJ5xICFF95Z5HfQ,Kathmandu Tandoori House,kathmandu-tandoori-house-madrid,"[{'alias': 'indpak', 'title': 'Indian'}, {'ali...",Indian Pakistani Himalayan/Nepalese,40.474260,-3.578590,"Calle Acuario, 11, 28042 Madrid, Spain",+34911269719,5.0,21,Great option for Indian and nepali food. The c...,https://www.yelp.com/biz/kathmandu-tandoori-ho...,https://s3-media2.fl.yelpcdn.com/bphoto/sS8gTX...,False,None,None,[],None,None,None,None,None,None,None
1,oPERop8gNCRC9LmbaXV46w,La General,la-general-madrid,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",Pizza Bakeries,40.470620,-3.580680,"Calle Planeta, 3, 28042 Madrid, Spain",+34916035701,5.0,12,"Man, even when looking for good eats internat...",https://www.yelp.com/biz/la-general-madrid?adj...,https://s3-media2.fl.yelpcdn.com/bphoto/Nq9scW...,False,None,None,[],None,None,None,None,None,None,None
2,4NyuSViM9LOFVYuMTZRmTw,Al Traste Fun And Food,al-traste-fun-and-food-madrid,"[{'alias': 'burgers', 'title': 'Burgers'}, {'a...",Burgers Pizza Sandwiches,40.470543,-3.580493,"Avenida General, 47, 28042 Madrid, Spain",+34913293799,4.0,20,For me the best place in the area. \nReally go...,https://www.yelp.com/biz/al-traste-fun-and-foo...,https://s3-media2.fl.yelpcdn.com/bphoto/ORQsb1...,False,€€,None,[],None,None,None,None,None,None,None


### **CHECK RESULTS**

In [269]:
map_madrid = folium.Map(location=[40.427919,-3.680877], zoom_start=14)

for (index, row) in df_distrito.iterrows():
    folium.Marker(location = [row.loc["latitude"], row.loc["longitude"]],
    #popup = row.loc["rotulo"] + " " + row["desc_distrito_local"],
    tooltip = "click").add_to(map_madrid)

map_madrid

In [270]:
df_distrito.to_csv("../data/barajas.csv")

In [271]:
'''
    0{"district": "Centro", "latitude": 40.415363, "longitude": -3.707398},
    1{"district": "Arganzuela", "latitude": 40.398068, "longitude": -3.693734},
    2{"district": "Retiro", "latitude": 40.411894, "longitude": -3.676574},
    {"district": "Salamanca", "latitude": 40.427919, "longitude": -3.680877},
    {"district": "Chamartín", "latitude": 40.462428, "longitude": -3.676057},
    {"district": "Tetuán", "latitude": 40.466676, "longitude": -3.698984},
    {"district": "Chamberí", "latitude": 40.436247, "longitude": -3.703830},
    {"district": "Fuencarral-El Pardo", "latitude": 40.515923, "longitude": -3.785586},
    {"district": "Moncloa-Aravaca", "latitude": 40.439883, "longitude": -3.744509},
    {"district": "Latina", "latitude": 40.388981, "longitude": -3.753525},
    {"district": "Carabanchel", "latitude": 40.377818, "longitude": -3.751209},
    {"district": "Usera", "latitude": 40.383553, "longitude": -3.706070},
    {"district": "Puente de Vallecas", "latitude": 40.389491, "longitude": -3.669062},
    {"district": "Moratalaz", "latitude": 40.407251, "longitude": -3.644992},
    {"district": "Ciudad Lineal", "latitude": 40.448431, "longitude": -3.650495},
    {"district": "Hortaleza", "latitude": 40.474457, "longitude": -3.640892},
    {"district": "Villaverde", "latitude": 40.345610, "longitude": -3.695956},
    {"district": "Villa de Vallecas", "latitude": 40.367040, "longitude": -3.612046},
    {"district": "Vicálvaro", "latitude": 40.396584, "longitude": -3.576622},
    {"district": "San Blas-Canillejas", "latitude": 40.428918, "longitude": -3.609064},
    {"district": "Barajas", "latitude": 40.477628, "longitude": -3.579394}
    '''

'\n    0{"district": "Centro", "latitude": 40.415363, "longitude": -3.707398},\n    1{"district": "Arganzuela", "latitude": 40.398068, "longitude": -3.693734},\n    2{"district": "Retiro", "latitude": 40.411894, "longitude": -3.676574},\n    {"district": "Salamanca", "latitude": 40.427919, "longitude": -3.680877},\n    {"district": "Chamartín", "latitude": 40.462428, "longitude": -3.676057},\n    {"district": "Tetuán", "latitude": 40.466676, "longitude": -3.698984},\n    {"district": "Chamberí", "latitude": 40.436247, "longitude": -3.703830},\n    {"district": "Fuencarral-El Pardo", "latitude": 40.515923, "longitude": -3.785586},\n    {"district": "Moncloa-Aravaca", "latitude": 40.439883, "longitude": -3.744509},\n    {"district": "Latina", "latitude": 40.388981, "longitude": -3.753525},\n    {"district": "Carabanchel", "latitude": 40.377818, "longitude": -3.751209},\n    {"district": "Usera", "latitude": 40.383553, "longitude": -3.706070},\n    {"district": "Puente de Vallecas", "lati